In [53]:
pip install torch transformers scikit-learn


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [41]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch
import pandas as pd

# Load pre-trained DistilBERT
MODEL_NAME = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

# Load example dataset (replace with your fintech/healthcare dataset)
# Sample structure: {'text': [description], 'label': [0 for non-fraud, 1 for fraud]}
data = pd.DataFrame({
    "text": [
        "Unauthorized transaction on your credit card",
        "Routine check-up note with no abnormalities",
        "Large withdrawal from an unusual location",
        "Patient shows irregular drug prescription history",
        "Normal transaction at a grocery store",
    ],
    "label": [1, 0, 1, 1, 0]
})

# Tokenize data
def tokenize_data(texts, labels):
    tokens = tokenizer(texts.tolist(), padding=True, truncation=True, max_length=512, return_tensors="pt")
    tokens["labels"] = torch.tensor(labels.tolist())
    return tokens

# Train-test split
train_texts, test_texts, train_labels, test_labels = train_test_split(data['text'], data['label'], test_size=0.2)
train_data = tokenize_data(train_texts, train_labels)
test_data = tokenize_data(test_texts, test_labels)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [43]:
# Define a custom evaluation metric
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

In [45]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    eval_strategy="epoch"
)

In [47]:
# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    processing_class = tokenizer,
    compute_metrics=compute_metrics
)

In [49]:
# Train the model
trainer.train()

AttributeError: 'list' object has no attribute 'keys'

In [51]:
# Evaluate the model
results = trainer.evaluate()
print("Test Results:", results)

IndexError: list index out of range

In [1]:
from transformers import AutoTokenizer
import pandas as pd
from sklearn.model_selection import train_test_split
import torch

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b", use_fast=False)

# Example dataset
data = pd.DataFrame({
    "text": [
        "Unauthorized transaction on your credit card",
        "Routine check-up note with no abnormalities",
        "Large withdrawal from an unusual location",
        "Patient shows irregular drug prescription history",
        "Normal transaction at a grocery store",
    ],
    "label": [1, 0, 1, 1, 0]  # 1 for Fraud, 0 for Non-Fraud
})

# Train-test split
train_texts, test_texts, train_labels, test_labels = train_test_split(
    data['text'], data['label'], test_size=0.2
)

# Tokenize text
def tokenize_texts(texts, labels):
    tokens = tokenizer(
        texts.tolist(),
        padding=True,
        truncation=True,
        max_length=256,
        return_tensors="pt"
    )
    tokens["labels"] = torch.tensor(labels.tolist())
    return tokens

train_data = tokenize_texts(train_texts, train_labels)
test_data = tokenize_texts(test_texts, test_labels)


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Llama-2-7b.
401 Client Error. (Request ID: Root=1-674d1670-54bb29106f089c542544f865;0547ffd6-80fb-4085-9295-6824907bebf1)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-2-7b/resolve/main/config.json.
Access to model meta-llama/Llama-2-7b is restricted. You must have access to it and be authenticated to access it. Please log in.